### 네이버 연관 검색어 수집
- 정적(static) 웹페이지 데이터 수집
- BeautifulSoup을 이용하여 HTML 문자열 데이터 parsing

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
# 1. 웹페이지 분석 : URL

In [ ]:
query = "삼성전자"
url = f"https://search.naver.com/search.naver?query={query}"
print(url)

https://search.naver.com/search.naver?query=삼성전자


In [ ]:
# 2. request(URL) > response : str(html)

In [ ]:
response = requests.get(url)
response

<Response [200]>

In [ ]:
response.text[:250]

'<!doctype html> <html lang="ko"> <head> <meta charset="utf-8"> <meta name="referrer" content="always">  <meta name="format-detection" content="telephone=no,address=no,email=no"> <meta name="viewport" content="width=device-width,initial-scale=1.0,maxi'

In [ ]:
# 3. str(html) > bs object

In [ ]:
dom = BeautifulSoup(response.text, "html.parser")
type(dom)

bs4.BeautifulSoup

In [ ]:
# 4. bs object > .select(css-selector), .select_one(css-selector) > str(text)

In [ ]:
# 10개의 엘리먼트들 선택
# select() : list(Tag, Tag)
# select_one() : Tag
elements = dom.select(".lst_related_srch > .item")
len(elements)

10

In [ ]:
element = elements[0]
keyword = element.select_one(".tit").text
keyword

'삼성전자주가'

In [ ]:
link = element.select_one("a").get("href")
link

'?where=nexearch&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%EC%A3%BC%EA%B0%80&ie=utf8&sm=tab_she&qdt=0'

In [ ]:
element.text.strip()

'삼성전자주가'

In [ ]:
# 각각의 엘리먼트에서 text 데이터 수집

In [ ]:
keywords = []
for element in elements:
    keyword = element.text.strip()
    keywords.append(keyword)
print(keywords)

['삼성전자주가', '삼성전자 배당금', '삼성전자 성과급', '삼성전자 주식', '삼성전자 세일 페스타', '삼성전자서비스', '오늘 삼성전자 주가', '삼성 전자레인지', '삼성전자 배당금 지급일', '삼성전자 opi']


In [ ]:
keywords = [element.text.strip() for element in elements]
print(keywords)

['삼성전자주가', '삼성전자 배당금', '삼성전자 성과급', '삼성전자 주식', '삼성전자 세일 페스타', '삼성전자서비스', '오늘 삼성전자 주가', '삼성 전자레인지', '삼성전자 배당금 지급일', '삼성전자 opi']


In [ ]:
# 5. str(text) > DataFrame

In [ ]:
df = pd.DataFrame({"keywors": keywords})
df["query"] = query
df.tail(2)

,keywors,query
8,삼성전자 배당금 지급일,삼성전자
9,삼성전자 opi,삼성전자


In [ ]:
# 현재 시간 데이터 추가

In [ ]:
from datetime import datetime

In [ ]:
now = datetime.now()
now = now.strftime("%Y-%m-%d %H:%M")
now

'2023-02-05 23:02'

In [ ]:
df["date_time"] = now
df.tail(2)

,keywors,query,date_time
8,삼성전자 배당금 지급일,삼성전자,2023-02-05 23:02
9,삼성전자 opi,삼성전자,2023-02-05 23:02


In [ ]:
# query를 입력하면 데이터 프레임을 출력하는 함수
def naver_relate_keyword(query):

    url = f"https://search.naver.com/search.naver?query={query}"
    response = requests.get(url)
    dom = BeautifulSoup(response.text, "html.parser")
    elements = dom.select(".lst_related_srch > .item")
    keywords = [element.text.strip() for element in elements]

    df = pd.DataFrame({"keywors": keywords})
    df["query"] = query

    now = datetime.now()
    now = now.strftime("%Y-%m-%d %H:%M")
    df["date_time"] = now

    return df

In [ ]:
query = "삼성전자"
df = naver_relate_keyword(query)
df.tail(2)

,keywors,query,date_time
8,삼성전자 배당금 지급일,삼성전자,2023-02-05 23:02
9,삼성전자 opi,삼성전자,2023-02-05 23:02


In [ ]:
dfs = []
queries = ["삼성전자", "LG전자"]

for query in queries:
    print(query, end=" ")
    df = naver_relate_keyword(query)
    dfs.append(df)

result = pd.concat(dfs, ignore_index=True)
result

삼성전자 LG전자 

,keywors,query,date_time
0,삼성전자주가,삼성전자,2023-02-05 23:02
1,삼성전자 배당금,삼성전자,2023-02-05 23:02
2,삼성전자 성과급,삼성전자,2023-02-05 23:02
3,삼성전자 주식,삼성전자,2023-02-05 23:02
4,삼성전자 세일 페스타,삼성전자,2023-02-05 23:02
5,삼성전자서비스,삼성전자,2023-02-05 23:02
6,오늘 삼성전자 주가,삼성전자,2023-02-05 23:02
7,삼성 전자레인지,삼성전자,2023-02-05 23:02
8,삼성전자 배당금 지급일,삼성전자,2023-02-05 23:02
9,삼성전자 opi,삼성전자,2023-02-05 23:02
